In [1]:
!pip install tensorflow==2.9.1
!pip install keras==2.9.0

     |████████████████████████████████| 511.7 MB 3.7 kB/s  eta 0:00:01
     |████████████████████████████████| 126 kB 43.1 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 42.5 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 42.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 41.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 41.0 MB/s eta 0:00:01
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: keras
    Found exis

IMPORTING PACKAGES

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import VGG16
import cv2
from tensorflow.keras.models import load_model
import numpy as np


IMPORTING TRAINING DATA

In [24]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e9a3d7cf91cf4bd0baf7ab4703a8d4df = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='7cp13pz8yd9to4kxml25Jjyb1MGFNXZn8TS5aplui_J3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_e9a3d7cf91cf4bd0baf7ab4703a8d4df.get_object(Bucket='insuranceproject-donotdelete-pr-ugqohp6gzqcw3h', Key='Dataset1.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [25]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
!pwd

/home/wsuser/work


In [26]:
image_generator=ImageDataGenerator(vertical_flip=False,horizontal_flip=True,shear_range=0.1,zoom_range=0.1,rescale=1/255,brightness_range=(0.2,0.8))
X_train1=image_generator.flow_from_directory(target_size=(224,224),
                                    directory="/home/wsuser/work/Dataset1/training1",
                                    class_mode="categorical",
                                   batch_size=10,
                                            subset="training")

Found 979 images belonging to 3 classes.


IMPORTING TESTING DATA

In [27]:
image_generator_1=ImageDataGenerator(vertical_flip=False,horizontal_flip=True,shear_range=0.1,zoom_range=0.1,rescale=1/255,brightness_range=(0.2,0.8))
X_test1=image_generator_1.flow_from_directory(target_size=(224,224),
                                    directory="/home/wsuser/work/Dataset1/validation1",
                                    class_mode="categorical",
                                   batch_size=10,
                                )

Found 171 images belonging to 3 classes.


INITIALIZING MODEL

In [28]:
vgg16=VGG16(include_top=False,input_shape=(224,224,3),weights='imagenet')
for i in vgg16.layers:
  i.trainable=False

ADD FLATTEN LAYER

In [29]:
flatten_layer=Flatten()(vgg16.output)

ADDING DENSE LAYER

In [30]:
output=Dense(3,activation="softmax")(flatten_layer)

BUILDING MODEL

In [31]:
model1=Model(inputs=vgg16.input,outputs=output)
model1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

INITILAIZE LEARNING PARAMETERS

In [34]:
model1.compile(loss=CategoricalCrossentropy(),
              optimizer=Adam(epsilon=0.001),
              metrics=["acc"])

FITTING DATA TO THE MODEL

In [35]:
model1.fit(X_train1,validation_data=X_test1,epochs=15,steps_per_epoch=45,validation_steps=9)

Epoch 1/15
45/45 [==============================] - 194s 4s/step - loss: 1.3399 - acc: 0.4477 - val_loss: 0.8752 - val_acc: 0.5222
Epoch 2/15
45/45 [==============================] - 193s 4s/step - loss: 0.9263 - acc: 0.5924 - val_loss: 1.1384 - val_acc: 0.5111
Epoch 3/15
45/45 [==============================] - 193s 4s/step - loss: 0.8340 - acc: 0.6503 - val_loss: 0.9405 - val_acc: 0.5556
Epoch 4/15
45/45 [==============================] - 193s 4s/step - loss: 0.7957 - acc: 0.6637 - val_loss: 0.8900 - val_acc: 0.5778
Epoch 5/15
45/45 [==============================] - 194s 4s/step - loss: 0.8085 - acc: 0.6800 - val_loss: 0.8961 - val_acc: 0.6111
Epoch 6/15
45/45 [==============================] - 194s 4s/step - loss: 0.7431 - acc: 0.7089 - val_loss: 0.9167 - val_acc: 0.6000
Epoch 7/15
45/45 [==============================] - 193s 4s/step - loss: 0.6848 - acc: 0.7111 - val_loss: 1.0671 - val_acc: 0.5222
Epoch 8/15
45/45 [==============================] - 193s 4s/step - loss: 0.5840 - a

SAVING THE MODEL

In [36]:
model1.save("LevelModel.h5")

LOAD MODEL

In [37]:
!tar -zcvf level_model.tgz Model1.h5


tar: Model1.h5: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [38]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 16.4 MB/s eta 0:00:01


In [39]:
cred={"url":"https://us-south.ml.cloud.ibm.com",
 "apikey":"F-FjystdF3aQ-G67Twcfyk0SRAukYiIFiB5g9QzJNkKT"}

In [40]:
from ibm_watson_machine_learning import APIClient

In [41]:
client=APIClient(cred)

In [42]:
def guid_from_space_name(client, space_name): 
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [43]:
space_uid=guid_from_space_name (client, 'Models') 
print("Space UID= " + space_uid)

Space UID= 11cba668-4b75-436d-af79-82c7d3905230


In [44]:
client.set.default_space(space_uid)


'SUCCESS'

In [45]:
client.software_specifications.list()


-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [50]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")


In [1]:
model_details=client.repository.store_model(model='level_model.tgz',meta_props={client.repository.ModelMetaNames.NAME:"CNN1", 
                                                                                   client.repository.ModelMetaNames.TYPE: "keras_2.2.4", 
                                                                                   client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid})
model_id=client.repository.get_model_uid(model_details)

NameError: name 'client' is not defined

In [ ]:
def detect(frame):
    img = cv2.resize(frame, (224, 224))
    if(np.max(img) > 1):
        img = img/255.0
    img = np.array([img])
    prediction = model.predict(img)
    label = ["minor", "moderate", "severe"]
    preds = label[np.argmax(prediction)]
    return preds